In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#import sys
#!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

# just set the seed for the random number generator
#np.random.seed(107)

In [ ]:
# Change...
pair = 'BTCUSDT'

# interval used to compute indicators and backtest analysis
# valid strategy intervals - 1min, 3min, 5min, 15min, 30min, 1H, 2H, 4H, 6H, 8H, 12H, 1D, 3D, 1W, 1M
strategy_interval = '1D'

# date to start/end analysis
# Use '' for not to limit the data
# 6 months before
#dt_start_analysis = date.today() - relativedelta(months=6)
# 1 year before
dt_start_analysis = date.today() - relativedelta(months=12)
dt_end_analysis = ''


In [ ]:
from binance_utils import init
from trade_utils import get_data

client = init()
data = get_data(client, pair, strategy_interval)

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.sample(10)

In [ ]:
from technical_indicator_utils import get_sma, get_ema, get_macd, get_rsi, get_adx, get_bbands, get_rvi, get_momentum
from trade_utils import get_num_daily_bars

daily_bars = get_num_daily_bars(data)

# Momentum
data['Momentum'] = get_sma(get_momentum(data['ClosePrice'], daily_bars*14), 14)

# Calculate returns
data['Returns'] = data['ClosePrice'].pct_change() * 100 # pct_change() = percent change from previous price
data['Returns5'] = data['ClosePrice'].pct_change(5) * 100 # change from 5 previous price
data['DailyReturns'] = data['ClosePrice'].pct_change(daily_bars) * 100 # daily change
data['MonthlyReturns'] = data['ClosePrice'].pct_change(daily_bars*30) * 100 # monthly change

# Volatility
data['MonthlyVolatility'] = data.rolling(daily_bars*30, min_periods = daily_bars*30)['Returns'].std()

# SMA
data['SMA5'] = get_sma(data['ClosePrice'], 5)
data['SMA10'] = get_sma(data['ClosePrice'], 10)
data['SMA20'] = get_sma(data['ClosePrice'], 20)
data['SMA30'] = get_sma(data['ClosePrice'], 30)
data['SMA50'] = get_sma(data['ClosePrice'], 50)
data['SMA100'] = get_sma(data['ClosePrice'], 100)
data['SMA200'] = get_sma(data['ClosePrice'], 200)

# EMA
data['EMA5'] = get_ema(data['ClosePrice'], 5)
data['EMA10'] = get_ema(data['ClosePrice'], 10)
data['EMA20'] = get_ema(data['ClosePrice'], 20)
data['EMA30'] = get_ema(data['ClosePrice'], 30)
data['EMA50'] = get_ema(data['ClosePrice'], 50)
data['EMA100'] = get_ema(data['ClosePrice'], 100)
data['EMA200'] = get_ema(data['ClosePrice'], 200)

# MACD
data['MACD'], data['MACDSignal'], data['MACDHist'] = get_macd(data['ClosePrice'])

# RVI
data['RVI'], data['RVISignal'] = get_rvi(data['OpenPrice'], data['ClosePrice'], data['LowPrice'], data['HighPrice'])

# RSI
data['RSI'] = get_rsi(data.ClosePrice)
data['RSI_10'] = get_rsi(data.ClosePrice, period=10)

# ADX
data['DI+'], data['DI-'], data['ADX'] = get_adx(data['HighPrice'], data['LowPrice'], data['ClosePrice'])

# Bollinger Bands
data['UpperBBand'], data['MidiBBand'], data['LowerBBand'] = get_bbands(data['ClosePrice'])

In [ ]:
#limit data to plot
if dt_end_analysis == '' and dt_start_analysis != '':
    data = data[dt_start_analysis:]
elif dt_end_analysis != '' and dt_start_analysis == '':
    data = data[:dt_end_analysis]
elif dt_end_analysis != '' and dt_start_analysis != '':
    data = data[dt_start_analysis:dt_end_analysis]

In [ ]:
from technical_indicator_utils import get_fibonacci_retracement_levels

# Fibonacci retracement
fib_levels = get_fibonacci_retracement_levels(data['ClosePrice'].max(), data['ClosePrice'].min())

In [ ]:
# Plot the histogram
data['Returns'].hist(bins=100, color='red')
plt.ylabel('Frequency')
plt.xlabel('Return')
plt.title('Return histogram')
plt.show()

In [ ]:
#plt.style.use('fivethirtyeight')

In [ ]:
fig, (ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8) = plt.subplots(nrows=8, sharex=True, figsize=(14, 28), gridspec_kw = {'height_ratios':[3, 3, 3, 1, 1, 1, 1, 1]})

# Price x SMA
ax1.plot(data.index, data[['ClosePrice', 'SMA50', 'SMA100']])
# Fibonacci retracement
ax1.axhspan(fib_levels['level1'], fib_levels['price_min'], alpha=0.4, color='lightsalmon')
ax1.axhspan(fib_levels['level2'], fib_levels['level1'], alpha=0.5, color='palegoldenrod')
ax1.axhspan(fib_levels['level3'], fib_levels['level2'], alpha=0.5, color='palegreen')
ax1.axhspan(fib_levels['price_max'], fib_levels['level3'], alpha=0.5, color='powderblue')
ax1.set_ylabel('Price - SMA')

# Price x EMA
ax2.plot(data.index, data[['ClosePrice', 'EMA50', 'EMA100']])
# Fibonacci retracement
ax2.axhspan(fib_levels['level1'], fib_levels['price_min'], alpha=0.4, color='lightsalmon')
ax2.axhspan(fib_levels['level2'], fib_levels['level1'], alpha=0.5, color='palegoldenrod')
ax2.axhspan(fib_levels['level3'], fib_levels['level2'], alpha=0.5, color='palegreen')
ax2.axhspan(fib_levels['price_max'], fib_levels['level3'], alpha=0.5, color='powderblue')
ax2.set_ylabel('Price - EMA')

# Bollinger Bands
ax3.plot(data['ClosePrice'], label='Price')
ax3.plot(data['UpperBBand'], label='Upper band')
ax3.plot(data['MidiBBand'], label='Middle band')
ax3.plot(data['LowerBBand'], label='Lower band')
ax3.legend()
ax3.set_ylabel('Bollinger Bands')

# MACD
ax4.plot(data.index, data['MACD'], label='MACD', color = 'red')
ax4.bar(data.index, data['MACDHist'], label='Hist', color="purple", width = timedelta(hours=1))
ax4.plot(data.index, data['MACDSignal'], label='Signal', color='blue')
ax4.legend()
ax4.set_ylabel('MACD')

# RVI
ax5.plot(data.index, data['RVI'], label='RVI', color = 'red')
ax5.plot(data.index, data['RVISignal'], label='Signal', color='blue')
ax5.legend()
ax5.set_ylabel('RVI')

# RSI
ax6.plot(data.index, [70] * len(data.index), label='overbought')
ax6.plot(data.index, [30] * len(data.index), label='oversold')
ax6.plot(data.index, data['RSI'], label='RSI')
ax6.legend()
ax6.set_ylabel('RSI')

# ADX
ax7.plot(data.index, data[['DI+', 'DI-']])
ax7.plot(data.index, data[['ADX']], color='purple', linestyle='dashed')
ax7.plot(data.index, [25] * len(data.index), color = 'red')
ax7.set_ylabel('ADX')

# Volume
# make bar plots and color differently depending on up/down for the day
pos = data['OpenPrice'] - data['ClosePrice'] < 0
neg = data['OpenPrice'] - data['ClosePrice'] > 0
ax8.bar(data.index[pos], data[pos]['Volume'], color='green', width = timedelta(hours=1))
ax8.bar(data.index[neg], data[neg]['Volume'], color='red', width = timedelta(hours=1))

#ax7.bar(data.index, data['Volume'] / 1000000, width = timedelta(hours=4))
ax8.set_ylabel('Volume')

fig.tight_layout()

## Backtest Strategy

In [ ]:
import bt

# Get the price data
price_data = data[['ClosePrice']].copy()

In [ ]:
from strategy_utils import get_cross_signal
from backtest_utils import signal_strategy

signal = get_cross_signal(data[['SMA50']].copy(), data[['SMA200']].copy())
bt_sma_cross = signal_strategy(price_data.copy(), signal, 'sma_cross')

In [ ]:
from strategy_utils import get_cross_signal
from backtest_utils import signal_strategy

signal = get_cross_signal(data[['EMA50']].copy(), data[['EMA200']].copy())
bt_ema_cross = signal_strategy(price_data.copy(), signal, 'ema_cross')

In [ ]:
from strategy_utils import get_rsi_signal
from backtest_utils import signal_strategy

#signal = get_rsi_signal(data[['RSI']].copy())
signal = get_rsi_signal(data[['RSI']].copy(), overbought_value=70, oversold_value=30)

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
plt.xticks(rotation=45)

plt.plot(combined_df['Price'])
plt.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
plt.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')

plt.title('Price Chart & Historical Trades', fontweight="bold")
plt.legend()
plt.show()

In [ ]:
bt_rsi = signal_strategy(price_data.copy(), signal, 'rsi')

bt_result = bt.run(bt_rsi)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_rsi_return_signal
from backtest_utils import signal_strategy

overbought_value = 80
oversold_value = 25

signal = get_rsi_return_signal(data[['RSI']].copy(), overbought_value=overbought_value, oversold_value=oversold_value)

bt_rsi_return = signal_strategy(price_data.copy(), signal, 'rsi_return')
bt_result = bt.run(bt_rsi_return)

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(20, 8))

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(data.index, [overbought_value] * len(data.index), label='overbought')
ax2.plot(data.index, [oversold_value] * len(data.index), label='oversold')
ax2.plot(data.index, data['RSI'], label='RSI')
ax2.legend()
ax2.set_ylabel('RSI')

ax3.plot(bt_result.prices)
ax3.set_ylabel('Result')

plt.xticks(rotation=45)
plt.show()

In [ ]:
bt_result.get_transactions()

In [ ]:
from strategy_utils import get_inverted_rsi_signal
from backtest_utils import signal_strategy

overbought_value = 70
signal = get_inverted_rsi_signal(data[['RSI']].copy(), overbought_value=overbought_value)

bt_inverted_rsi = signal_strategy(price_data.copy(), signal, 'rsi_inverted')
bt_result = bt.run(bt_inverted_rsi)

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(20, 8))

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(data.index, [overbought_value] * len(data.index), label='overbought')
#ax2.plot(data.index, [oversold_value] * len(data.index), label='oversold')
ax2.plot(data.index, data['RSI'], label='RSI')
ax2.legend()
ax2.set_ylabel('RSI')

ax3.plot(bt_result.prices)
ax3.set_ylabel('Result')

plt.xticks(rotation=45)
plt.show()

In [ ]:
from strategy_utils import get_rsi_adx_signal
from backtest_utils import signal_strategy

signal = get_rsi_adx_signal(data[['RSI']].copy(), data[['ADX']].copy(), data[['DI+']].copy(), data[['DI-']].copy(), overbought_value=70.0, oversold_value=30.0)

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_rsi_adx = signal_strategy(price_data.copy(), signal, 'rsi_adx')

bt_result = bt.run(bt_rsi_adx)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_rsi_plus_signal
from backtest_utils import signal_strategy

signal = get_rsi_plus_signal(data[['RSI']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_rsi_plus = signal_strategy(price_data.copy(), signal, 'rsi_plus')

bt_result = bt.run(bt_rsi_plus)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_macd_signal
from backtest_utils import signal_strategy

signal = get_macd_signal(data[['MACDSignal']].copy(), data[['MACD']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
plt.xticks(rotation=45)

plt.plot(combined_df['Price'])
plt.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
plt.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')

plt.title('Price Chart & Historical Trades', fontweight="bold")
plt.legend()
plt.show()

In [ ]:
bt_macd = signal_strategy(price_data.copy(), signal, 'macd')

bt_result = bt.run(bt_macd)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_macd_rvi_signal
from backtest_utils import signal_strategy

signal = get_macd_rvi_signal(data[['MACDSignal']].copy(), data[['MACD']].copy(), data[['RVISignal']].copy(), data[['RVI']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_macd_rvi = signal_strategy(price_data.copy(), signal, 'macd_rvi')

bt_result = bt.run(bt_macd_rvi)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_sma_macd_signal
from backtest_utils import signal_strategy

signal = get_sma_macd_signal(data[['ClosePrice']].copy(), data[['SMA50']].copy(), data[['SMA100']].copy(), data[['MACD']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_sma_macd = signal_strategy(price_data.copy(), signal, 'sma_macd')

bt_result = bt.run(bt_sma_macd)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_adx_macd_signal
from backtest_utils import signal_strategy

signal = get_adx_macd_signal(data[['MACD']].copy(), data[['DI+']].copy(), data[['DI-']].copy(), data[['ADX']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_adx_macd = signal_strategy(price_data.copy(), signal, 'adx_macd')

bt_result = bt.run(bt_adx_macd)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_bbands_signal
from backtest_utils import signal_strategy

signal = get_bbands_signal(data[['ClosePrice']].copy(), data[['UpperBBand']].copy(), data[['LowerBBand']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
bt_bbands = signal_strategy(price_data.copy(), signal, 'bbands')

bt_result = bt.run(bt_bbands)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from strategy_utils import get_rsi_bbands_signal
from backtest_utils import signal_strategy

signal = get_rsi_bbands_signal(data[['ClosePrice']].copy(), data[['UpperBBand']].copy(), data[['LowerBBand']].copy(), data[['RSI_10']].copy())

# Merge the data
combined_df = bt.merge(signal, price_data)
combined_df.columns = ['signal', 'Price']
combined_df.plot(secondary_y=['signal'])
plt.show()

In [ ]:
title = 'RSI + BBAnds Strategy ' + pair + '-' + strategy_interval

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(20, 8))

ax1.plot(combined_df['Price'])
ax1.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
ax1.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')
ax1.legend()
ax1.set_ylabel('Price')

ax2.plot(data['ClosePrice'], label='Price')
ax2.plot(data['UpperBBand'], label='Upper band')
ax2.plot(data['MidiBBand'], label='Middle band')
ax2.plot(data['LowerBBand'], label='Lower band')
ax2.legend()
ax2.set_ylabel('Bollinger Bands')

ax3.plot(data.index, [70] * len(data.index), label='overbought')
ax3.plot(data.index, [30] * len(data.index), label='oversold')
ax3.plot(data.index, data['RSI_10'], label='RSI')
ax3.legend()
ax3.set_ylabel('RSI')

plt.xticks(rotation=45)
plt.show()

In [ ]:
bt_rsi_bbands = signal_strategy(price_data.copy(), signal, 'rsi_bbands')

bt_result = bt.run(bt_rsi_bbands)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
from backtest_utils import signal_above_strategy, buy_and_hold_strategy

# Create signal strategy backtest
bt_sma5 = signal_above_strategy(price_data.copy(), data[['SMA5']].copy(), name='SMA5')
bt_sma10 = signal_above_strategy(price_data.copy(), data[['SMA10']].copy(), name='SMA10')
bt_sma20 = signal_above_strategy(price_data.copy(), data[['SMA20']].copy(), name='SMA20')
bt_sma30 = signal_above_strategy(price_data.copy(), data[['SMA30']].copy(), name='SMA30')
bt_sma50 = signal_above_strategy(price_data.copy(), data[['SMA50']].copy(), name='SMA50')
bt_sma100 = signal_above_strategy(price_data.copy(), data[['SMA100']].copy(), name='SMA100')

# Create benchmark strategy backtest
bt_benchmark = buy_and_hold_strategy(price_data.copy(), name='benchmark')

In [ ]:
# Run all backtests and plot the resutls
bt_results = bt.run(bt_sma5, bt_sma10, bt_sma20, bt_sma30, bt_sma50, bt_sma100, 
                    #bt_sma_cross, bt_ema_cross, bt_rsi, bt_rsi_adx, bt_rsi_plus, bt_macd, bt_sma_macd, bt_adx_macd, bt_bbands, bt_inverted_rsi, bt_macd_rvi,
                    bt_rsi, bt_rsi_adx, bt_rsi_plus, bt_macd, bt_sma_macd, bt_adx_macd, bt_bbands, bt_rsi_bbands, bt_inverted_rsi, bt_macd_rvi, bt_rsi_return,
                    bt_benchmark)
# Plot results
bt_results.plot(title='Strategy benchmarking')
plt.show()

In [ ]:
bt_results.display()

In [ ]:
# Obtain all backtest stats
resInfo = bt_results.stats
ordered_strategies = resInfo.loc['total_return'].sort_values(ascending=False)
print(ordered_strategies)

best_strategy = ''
if resInfo.loc['total_return'].sort_values(ascending=False).index[0] == 'benchmark':
    best_strategy = resInfo.loc['total_return'].sort_values(ascending=False).index[1]
else:
    best_strategy = resInfo.loc['total_return'].sort_values(ascending=False).index[0]

bt_best = bt_results.backtests[best_strategy]

In [ ]:
bt_results_best = bt.run(bt_benchmark, bt_best)
bt_results_best.plot(title='Strategy benchmarking')
plt.show()

In [ ]:
bt_results_best.get_transactions(strategy_name=best_strategy)

In [ ]:
# Plot the backtest result
title = 'Backtest result ' + pair + '-' + strategy_interval
bt_results.plot(title=title)
plt.show()

# Get the lookback returns
lookback_returns = bt_results.display_lookback_returns()
print(lookback_returns)

# Tests

In [ ]:
bt_results_test = bt.run(bt_benchmark, bt_sma50, bt_rsi_return)
title = 'Strategy benchmarking ' + pair + '-' + strategy_interval

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(20, 8))

ax1.plot(bt_results_test.prices)
ax1.legend(bt_results_test.prices.columns)
ax2.plot(data['MonthlyVolatility'])
ax2.axhline(4.0, color='red', linestyle='--')
ax3.plot(data['Momentum'])
ax3.axhline(0.0, color='red', linestyle='--')
#ax3.axhline(-0.1, color='red', linestyle='--')
plt.show()


In [ ]:
# Test combining signals
from strategy_utils import get_macd_signal, remove_repeated_signal
from backtest_utils import signal_strategy

treshold = 0.0
overbought_value = 75
oversold_value = 30

signal = price_data.copy()
signal['Momentum'] = data[['Momentum']].copy()
signal['rsi_return'] = get_rsi_return_signal(data[['RSI']].copy(), overbought_value=overbought_value, oversold_value=oversold_value)
signal['SMA50'] = get_cross_signal(price_data.copy(), data[['SMA50']].copy())

signal['signal'] = np.where(
    (signal['Momentum'] < treshold), signal['rsi_return'], signal['SMA50'])

signal = signal[['signal']].copy()
#remove repeated signals
signal = remove_repeated_signal(signal, 'signal')

# Merge the data
combined_df = bt.merge(signal[['signal']].copy(), price_data)
combined_df.columns = ['signal', 'Price']

plt.figure(figsize=(12,5))
plt.xticks(rotation=45)

plt.plot(combined_df['Price'])
plt.scatter(combined_df[combined_df['signal'] == 1.0].index, combined_df[combined_df['signal'] == 1.0]['Price'], label = 'Buy', marker='^', c='g')
plt.scatter(combined_df[combined_df['signal'] == -1.0].index, combined_df[combined_df['signal'] == -1.0]['Price'], label = 'Sell', marker='v', c='r')

plt.title('Price Chart & Historical Trades', fontweight="bold")
plt.legend()
plt.show()

In [ ]:
bt_rsir_sma50 = signal_strategy(price_data.copy(), signal, 'rsir_sma50')

bt_result = bt.run(bt_rsir_sma50)
# Plot the backtest result
bt_result.plot(title='Backtest result')
plt.show()

In [ ]:
data['Momentum'].plot()